In [4]:
import numpy as np
import gym
from collections import defaultdict

In [5]:
env = gym.make('GuessingGame-v0')

c:\users\victor a\appdata\local\programs\python\python38\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [149]:
print(env.action_space, env.observation_space)

Box(1,) Discrete(4)


In [141]:
action = np.array([10000])
env.step(action)
print(env.observation)

3


In [472]:
def generate_episode(env, Q, epsilon):
    state = env.reset()
    episode = []
    s = -1000
    e = 1000 
    while True:
        m = (s+e)/2
        #Actions are to go low, high, or reset/restart
        l = (s + m)/2
        h = (e + m)/2
        r = 0
        actions = [l, h, r]
        action = np.array([np.random.choice(actions, p = get_prob(Q[state], epsilon))])
        state, reward, done, info = env.step(action)
        if action == l:
            e = m
            categorical_action = 0
        elif action == h:
            s = m
            categorical_action = 1
        elif action == r:
            s,e = -1000, 1000
            categorical_action = 2
        episode.append((state, categorical_action, reward))
        if done:
            break
    return episode
        
def get_prob(Q_s, epsilon):
    probs = np.zeros(3) + (epsilon/3)
    best = np.argmax(Q_s)
    probs[best] = 1 - epsilon + (epsilon/3)
    return probs

def update_Q(env, Q, episode, alpha, gamma):
    states, actions, rewards = zip(*episode)
    discounts = np.array([gamma**i for i in range(len(rewards)+1)])

    for i, state in enumerate(states):
        Q_prime = Q[state][actions[i]]
        Q[state][actions[i]] = Q_prime + alpha*(sum(rewards[i:]*discounts[:-(i+1)]) - Q_prime)
    return Q

In [487]:
def GGMC(env, num_episodes = 600, alpha = 0.2, gamma = 0.99999999, epsilon_start= 1.0, epsilon_decay = 0.9999, epsilon_min = 0.2):
    nA = 3
    Q = defaultdict(lambda: np.zeros(3))
    epsilon = epsilon_start
    for i_episode in range(1, num_episodes + 1):
        if i_episode % 10 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
        epsilon = max(epsilon*epsilon_decay, epsilon_min)
        episode = generate_episode(env, Q, epsilon)
        Q = update_Q(env, Q, episode, alpha, gamma)
    print()
    print('Results')
    print('Action 0: Low, Action 1: High, Action 2: Reset')
    print()
    print('State 0')
    print(Q[0])
    print('Ordered Values:',np.argsort(Q[0]))
    print('Largest Value:',np.argmax(Q[0]))
    print()
    print('State 1')
    print(Q[1])
    print('Ordered Values:',np.argsort(Q[1]))
    print('Largest Value:',np.argmax(Q[1]))
    print()
    print('State 3')
    print(Q[3])
    print('Ordered Values:',np.argsort(Q[3]))
    print('Largest Value:',np.argmax(Q[3]))

In [490]:
GGMC(env)

Episode 600/600.
Results

State 0
[0. 0. 0.]
Ordered Values: [0 1 2]
Largest Value: 0

State 1
[1.69919043e-20 4.92394656e-13 6.47950761e-20]
Ordered Values: [0 2 1]
Largest Value: 1

State 3
[1.09636737e-19 2.33597139e-22 6.53485872e-19]
Ordered Values: [1 0 2]
Largest Value: 2


# Notes and Ending Remarks
I think I made a mistake to create the third action of reseting, because sometimes it has the largest q value. I tried changing the parameters a bunch but somehow the third one was still sometimes the highest value action. I even tried giving a negative reward upon choosing the third action and that somehow always made the third action the highest value. Anyways I should of just been more nature and made it only lower or higher and wont care if the episodes don't finish and give certain actions 0 value. I'm just glad that it seems to consistently choose the best policy if you disregrad the reset action. This did make me interested in doing evolutionary algorithms though!